In [1]:
import pandas as pd
import numpy as np
from preprocess import *
from svdpp import *

num_users = 943
num_items = 1682
ratings_train1 = pd.read_csv('train.csv')
arr_train1 = ratings_train1.to_numpy()

R1 = np.zeros((num_users,num_items))

for record in arr_train1:
    uid, mid, _, _ = record
    R1[uid-1][mid-1] = 1
    
mask = (R1>-1)
svdpp = SVDPP(R = R1)


In [2]:
import datetime
t1 = datetime.datetime.now()
loss_history = svdpp.train(R=R1,mask=mask,epochs=1000,lr=20,reg=0)
t2 = datetime.datetime.now()
print(t2-t1)

0:01:23.044838


In [3]:
for i in range(10):
    print(loss_history[i*10])

0.031226100348354883
0.028883188119658777
0.027388803832240335
0.02640295756407454
0.025731919491654834
0.025262348009157037
0.02492594938404555
0.024680258475405763
0.02449801260283508
0.02436115887073583


In [4]:
for j in range(10):
    print(loss_history[j*100+99])

0.024266550596611355
0.023934548165459875
0.02390631039282177
0.02390365271547163
0.02390328623256432
0.02390310158129791
0.02390289751284665
0.023902644763647583
0.023902327089124795
0.02390192594639028


In [5]:
recs = np.array(svdpp.test_bi()[1:])
reclists = recs.astype(np.int)
test_data = pd.read_csv('test.csv')
test_data.loc[0,'movie_id']
cnt =0
for i in range(num_users):
    if test_data.loc[i,'movie_id'] in reclists[i,:5,0]:
        cnt+=1
cnt

24

In [6]:
from metrics import dcg
from statistics import mean

l=list(test_data.movie_id)
l=[[x,x,x,x,x] for x in l]
ideal = np.array(l)
mean([dcg(3,x,5) for x in (reclists[:,:5,0]==ideal)])

0.01429395218387828